# UNSW2018 dataset

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = './UNSW2018/4image_final' 
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=data, transform=transform)
loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False)

In [3]:
# Feature extractor (consistent with the definition during training)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.noise = nn.Parameter(torch.randn(84))  # Feature confusion

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = nn.ReLU()(self.fc1(out))
        out = self.fc2(out) + self.noise  # Bottleneck layer and feature confusion
        return out

In [4]:
# Load the weights of the trained feature extractor
model_dir = './CIC/model'  
feature_extractor = FeatureExtractor().to(device)
feature_extractor.load_state_dict(torch.load(os.path.join(model_dir, 'feature_extractor_CIC.pth'), map_location=device))

<All keys matched successfully>

In [5]:
# Feature extraction and saving
def extract_and_save_features(feature_extractor, loader, device):
    feature_extractor.eval()
    features_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            features = feature_extractor(images)
            features_list.append(features.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())
    
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.array(labels_list)

    np.save('./CIC/features/features_UNSW2018.npy', features_array)
    np.save('./CIC/features/labels_UNSW2018.npy', labels_array)

In [6]:
os.makedirs('./CIC/features', exist_ok=True)
extract_and_save_features(feature_extractor, loader, device)

# Identification_UNSW2018

In [7]:
import torch.optim as optim
import time
import gc
import psutil
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# Load data from features.npy and labels.npy files
features = np.load('./CIC/features/features_UNSW2018.npy')
labels = np.load('./CIC/features/labels_UNSW2018.npy')

features = torch.tensor(features, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.long).to(device)

input_dim = features.shape[1]
num_classes = len(np.unique(labels.cpu().numpy()))

classifiers = [
    ("SimpleNN", SimpleNN(input_dim=input_dim, num_classes=num_classes).to(device))
]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [10]:
for name, classifier in classifiers:
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    kappas = []
    conf_matrix_sum = np.zeros((num_classes, num_classes))
    training_times = []
    testing_times = []
    memory_usages = []
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        classifier.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(classifier.parameters(), lr=0.001)
        
        # Record memory usage before training
        gc.collect()
        process = psutil.Process(os.getpid())
        start_memory = process.memory_info().rss / (1024 * 1024) 
        start_train_time = time.time()
        classifier.train()
        num_epochs = 10
        for epoch in range(num_epochs):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = classifier(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
        end_train_time = time.time()
        
        classifier.eval()
        with torch.no_grad():
            start_test_time = time.time()
            outputs = classifier(X_test)
            _, preds = torch.max(outputs, 1)
            end_test_time = time.time()
        
        accuracy = accuracy_score(y_test.cpu().numpy(), preds.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        recall = recall_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        kappa = cohen_kappa_score(y_test.cpu().numpy(), preds.cpu().numpy())
        conf_matrix = confusion_matrix(y_test.cpu().numpy(), preds.cpu().numpy(), labels=range(num_classes))
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        kappas.append(kappa)
        conf_matrix_sum += conf_matrix 
        training_times.append(end_train_time - start_train_time)
        testing_times.append(end_test_time - start_test_time)
        
        # Record memory usage after training and calculate the difference
        end_memory = process.memory_info().rss / (1024 * 1024)  
        memory_usages.append(end_memory - start_memory)
    
    print(f'\n{name} 10 fold cross validation results:')
    print(f'Average accuracy: {np.mean(accuracies) * 100:.2f}%')
    print(f'Average precision: {np.mean(precisions) * 100:.2f}%')
    print(f'Average Recall: {np.mean(recalls) * 100:.2f}%')
    print(f'Average F1: {np.mean(f1s):.2f}')
    print(f'Average Kappa: {np.mean(kappas):.2f}')
    print(f'Average train time: {np.mean(training_times):.2f} S')
    print(f'Average test time: {np.mean(testing_times):.3f} S')
    print(f'Average memory: {np.mean(memory_usages):.2f} MB')


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set


SimpleNN 10 fold cross validation results:
Average accuracy: 99.27%
Average precision: 99.29%
Average Recall: 99.27%
Average F1: 0.99
Average Kappa: 0.99
Average train time: 30.99 S
Average test time: 0.003 S
Average memory: 30.53 MB


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# UNSW2019 dataset

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = './UNSW2019/4image_final' 
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=data, transform=transform)
loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False)
# Feature extractor (consistent with the definition during training)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.noise = nn.Parameter(torch.randn(84))  # Feature confusion

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = nn.ReLU()(self.fc1(out))
        out = self.fc2(out) + self.noise  # Bottleneck layer and feature confusion
        return out
# Load the weights of the trained feature extractor
model_dir = './CIC/model'  
feature_extractor = FeatureExtractor().to(device)
feature_extractor.load_state_dict(torch.load(os.path.join(model_dir, 'feature_extractor_CIC.pth'), map_location=device))
# Feature extraction and saving
def extract_and_save_features(feature_extractor, loader, device):
    feature_extractor.eval()
    features_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            features = feature_extractor(images)
            features_list.append(features.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())
    
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.array(labels_list)

    np.save('./CIC/features/features_UNSW2019.npy', features_array)
    np.save('./CIC/features/labels_UNSW2019.npy', labels_array)
os.makedirs('./CIC/features', exist_ok=True)
extract_and_save_features(feature_extractor, loader, device)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x
# Load data from features.npy and labels.npy files
features = np.load('./CIC/features/features_UNSW2019.npy')
labels = np.load('./CIC/features/labels_UNSW2019.npy')

features = torch.tensor(features, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.long).to(device)

input_dim = features.shape[1]
num_classes = len(np.unique(labels.cpu().numpy()))

classifiers = [
    ("SimpleNN", SimpleNN(input_dim=input_dim, num_classes=num_classes).to(device))
]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for name, classifier in classifiers:
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    kappas = []
    conf_matrix_sum = np.zeros((num_classes, num_classes))
    training_times = []
    testing_times = []
    memory_usages = []
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        classifier.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(classifier.parameters(), lr=0.001)
        
        # Record memory usage before training
        gc.collect()
        process = psutil.Process(os.getpid())
        start_memory = process.memory_info().rss / (1024 * 1024) 
        start_train_time = time.time()
        classifier.train()
        num_epochs = 10
        for epoch in range(num_epochs):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = classifier(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
        end_train_time = time.time()
        
        classifier.eval()
        with torch.no_grad():
            start_test_time = time.time()
            outputs = classifier(X_test)
            _, preds = torch.max(outputs, 1)
            end_test_time = time.time()
        
        accuracy = accuracy_score(y_test.cpu().numpy(), preds.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        recall = recall_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        kappa = cohen_kappa_score(y_test.cpu().numpy(), preds.cpu().numpy())
        conf_matrix = confusion_matrix(y_test.cpu().numpy(), preds.cpu().numpy(), labels=range(num_classes))
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        kappas.append(kappa)
        conf_matrix_sum += conf_matrix 
        training_times.append(end_train_time - start_train_time)
        testing_times.append(end_test_time - start_test_time)
        
        # Record memory usage after training and calculate the difference
        end_memory = process.memory_info().rss / (1024 * 1024)  
        memory_usages.append(end_memory - start_memory)
    
    print(f'\n{name} 10 fold cross validation results:')
    print(f'Average accuracy: {np.mean(accuracies) * 100:.2f}%')
    print(f'Average precision: {np.mean(precisions) * 100:.2f}%')
    print(f'Average Recall: {np.mean(recalls) * 100:.2f}%')
    print(f'Average F1: {np.mean(f1s):.2f}')
    print(f'Average Kappa: {np.mean(kappas):.2f}')
    print(f'Average train time: {np.mean(training_times):.2f} S')
    print(f'Average test time: {np.mean(testing_times):.3f} S')
    print(f'Average memory: {np.mean(memory_usages):.2f} MB')



SimpleNN 10 fold cross validation results:
Average accuracy: 99.95%
Average precision: 99.94%
Average Recall: 99.95%
Average F1: 1.00
Average Kappa: 1.00
Average train time: 105.10 S
Average test time: 0.019 S
Average memory: 89.24 MB


# IMC dataset

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = './IMC/4image_final' 
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=data, transform=transform)
loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False)
# Feature extractor (consistent with the definition during training)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.noise = nn.Parameter(torch.randn(84))  # Feature confusion

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = nn.ReLU()(self.fc1(out))
        out = self.fc2(out) + self.noise  # Bottleneck layer and feature confusion
        return out
# Load the weights of the trained feature extractor
model_dir = './CIC/model'  
feature_extractor = FeatureExtractor().to(device)
feature_extractor.load_state_dict(torch.load(os.path.join(model_dir, 'feature_extractor_CIC.pth'), map_location=device))
# Feature extraction and saving
def extract_and_save_features(feature_extractor, loader, device):
    feature_extractor.eval()
    features_list = []
    labels_list = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            features = feature_extractor(images)
            features_list.append(features.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())
    
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.array(labels_list)

    np.save('./CIC/features/features_IMC.npy', features_array)
    np.save('./CIC/features/labels_IMC.npy', labels_array)
os.makedirs('./CIC/features', exist_ok=True)
extract_and_save_features(feature_extractor, loader, device)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x
# Load data from features.npy and labels.npy files
features = np.load('./CIC/features/features_IMC.npy')
labels = np.load('./CIC/features/labels_IMC.npy')

features = torch.tensor(features, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.long).to(device)

input_dim = features.shape[1]
num_classes = len(np.unique(labels.cpu().numpy()))

classifiers = [
    ("SimpleNN", SimpleNN(input_dim=input_dim, num_classes=num_classes).to(device))
]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for name, classifier in classifiers:
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    kappas = []
    conf_matrix_sum = np.zeros((num_classes, num_classes))
    training_times = []
    testing_times = []
    memory_usages = []
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        classifier.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(classifier.parameters(), lr=0.001)
        
        # Record memory usage before training
        gc.collect()
        process = psutil.Process(os.getpid())
        start_memory = process.memory_info().rss / (1024 * 1024) 
        start_train_time = time.time()
        classifier.train()
        num_epochs = 10
        for epoch in range(num_epochs):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = classifier(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
        end_train_time = time.time()
        
        classifier.eval()
        with torch.no_grad():
            start_test_time = time.time()
            outputs = classifier(X_test)
            _, preds = torch.max(outputs, 1)
            end_test_time = time.time()
        
        accuracy = accuracy_score(y_test.cpu().numpy(), preds.cpu().numpy())
        precision = precision_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        recall = recall_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average='weighted')
        kappa = cohen_kappa_score(y_test.cpu().numpy(), preds.cpu().numpy())
        conf_matrix = confusion_matrix(y_test.cpu().numpy(), preds.cpu().numpy(), labels=range(num_classes))
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        kappas.append(kappa)
        conf_matrix_sum += conf_matrix 
        training_times.append(end_train_time - start_train_time)
        testing_times.append(end_test_time - start_test_time)
        
        # Record memory usage after training and calculate the difference
        end_memory = process.memory_info().rss / (1024 * 1024)  
        memory_usages.append(end_memory - start_memory)
    
    print(f'\n{name} 10 fold cross validation results:')
    print(f'Average accuracy: {np.mean(accuracies) * 100:.2f}%')
    print(f'Average precision: {np.mean(precisions) * 100:.2f}%')
    print(f'Average Recall: {np.mean(recalls) * 100:.2f}%')
    print(f'Average F1: {np.mean(f1s):.2f}')
    print(f'Average Kappa: {np.mean(kappas):.2f}')
    print(f'Average train time: {np.mean(training_times):.2f} S')
    print(f'Average test time: {np.mean(testing_times):.3f} S')
    print(f'Average memory: {np.mean(memory_usages):.2f} MB')


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in 


SimpleNN 10 fold cross validation results:
Average accuracy: 96.72%
Average precision: 96.68%
Average Recall: 96.72%
Average F1: 0.96
Average Kappa: 0.96
Average train time: 6.30 S
Average test time: 0.005 S
Average memory: 4.21 MB


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
